# Coronavirus Data Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
spark.sparkContext.applicationId

'application_1636170550178_0037'

In [3]:
pd.options.display.max_columns = 35

### Create Variables for Queries

In [4]:
# Look back days variables
daysInterval = 365
print(daysInterval)
maxDays = 365
print(maxDays)
oneYearInterval=365
print(oneYearInterval)

# Username variable
user_id = 'rc047407'           ####<-------------------- change to your user id (Ex: tl068507)
userName = 'user_' + user_id
print(userName)

# Algorithm name variable
algorithmName='CoronavirusLengthOfStay_HighLevel'
print(algorithmName)

365
365
365
user_rc047407
CoronavirusLengthOfStay_HighLevel


In [5]:
stdVar = spark.sql(f"""
  select  round(std(h.length_of_stay),2) as std_length_of_stay
       
  from {userName}.{algorithmName}_Hospital h
  
  left join {userName}.{algorithmName}_Vaccine v
    on h.personid = v.personId
      and h.tenant = v.tenant

   where    
      datediff(h.serviceDate, h.dischargeDate) < 0
      and h.length_of_stay > 0
               
      and  ( datediff(h.serviceDate, v.vaccine_date) > 0
             and v.vaccine_date is not null)        
      
    
""")
stdValue_vac = stdVar.first()['std_length_of_stay']
print(stdValue_vac)

5.62


In [6]:
spark.sql(f"""
    select  age_range
          , vaccine_name
          , count(distinct personId) as count_of_vaccinated_patients
          , round(mean(length_of_stay),2) as mean_length_of_stay
        from (  
          select
              h.personId
              , length_of_stay
              , case 
                when taken_Moderna_vaccine = 1 then "Moderna"
                when taken_Pfizer_vaccine = 1 then "Pfizer"
                when taken_Janssen_vaccine = 1 then "Janssen"
              end as vaccine_name 
              , case
                when age >= 90 then "90+"
                when age >= 85 and age < 90 then "85-89"
                when age >= 80 and age < 85 then "80-85"
                when age >= 75 and age < 80 then "75-79"
                when age >= 70 and age < 75 then "70-74"
                when age >= 65 and age < 70 then "65-69"
                when age >= 60 and age < 65 then "60-65"
                when age >= 55 and age < 60 then "55-59"
                when age >= 50 and age < 55 then "50-54"
                when age >= 45 and age < 50 then "45-49"
                when age >= 40 and age < 45 then "40-45"
                when age >= 35 and age < 40 then "35-39"
                when age >= 30 and age < 35 then "30-34"
                when age >= 25 and age < 30 then "25-29"
                when age >= 20 and age < 25 then "20-25"
                when age >= 12 and age < 20 then "12-19"
                when age >= 0 and age < 12 then "0-12"
                else "unknown"
                end as age_range
              
        from {userName}.{algorithmName}_Hospital h
          
        join {userName}.{algorithmName}_Vaccine_Detail v
            on h.personid = v.personId
              and h.tenant = v.tenant
              and  ( datediff(h.serviceDate, v.vaccine_date) > 0
              and     v.vaccine_date is not null) 
              and ( taken_Moderna_vaccine = 1   
                    or taken_Pfizer_vaccine = 1  
                    or taken_Janssen_vaccine = 1 )

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}
              
              and deceased = 1

          ) subquery   
          
        group by 1, 2
        
        order by 1, 2
     
""").toPandas()

,age_range,vaccine_name,count_of_vaccinated_patients,mean_length_of_stay
0,55-59,Moderna,1,5.0
1,60-65,Moderna,1,3.0
2,65-69,Moderna,1,6.0
3,70-74,Pfizer,1,5.0
4,75-79,Moderna,1,4.0
5,80-85,Moderna,3,5.5
6,80-85,Pfizer,2,4.0
7,90+,Moderna,1,3.0


In [8]:
spark.sql(f"""
    select vaccine_name
          , count(distinct personId) as count_of_vaccinated_patients
          , round(mean(length_of_stay),2) as mean_length_of_stay
        from (  
          select
              h.personId
              , length_of_stay
              , case 
                when taken_Moderna_vaccine = 1 then "Moderna"
                when taken_Pfizer_vaccine = 1 then "Pfizer"
                when taken_Janssen_vaccine = 1 then "Janssen"
              end as vaccine_name 
              , case
                when age >= 90 then "90+"
                when age >= 85 and age < 90 then "85-89"
                when age >= 80 and age < 85 then "80-85"
                when age >= 75 and age < 80 then "75-79"
                when age >= 70 and age < 75 then "70-74"
                when age >= 65 and age < 70 then "65-69"
                when age >= 60 and age < 65 then "60-65"
                when age >= 55 and age < 60 then "55-59"
                when age >= 50 and age < 55 then "50-54"
                when age >= 45 and age < 50 then "45-49"
                when age >= 40 and age < 45 then "40-45"
                when age >= 35 and age < 40 then "35-39"
                when age >= 30 and age < 35 then "30-34"
                when age >= 25 and age < 30 then "25-29"
                when age >= 20 and age < 25 then "20-25"
                when age >= 12 and age < 20 then "12-19"
                when age >= 0 and age < 12 then "0-12"
                else "unknown"
                end as age_range
              
        from {userName}.{algorithmName}_Hospital h
          
        join {userName}.{algorithmName}_Vaccine_Detail v
            on h.personid = v.personId
              and h.tenant = v.tenant
              and  ( datediff(h.serviceDate, v.vaccine_date) > 0
              and     v.vaccine_date is not null) 
              and ( taken_Moderna_vaccine = 1   
                    or taken_Pfizer_vaccine = 1  
                    or taken_Janssen_vaccine = 1 )

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}
              
              and deceased = 1

          ) subquery   
          group by 1
          order by 1

""").toPandas()

,vaccine_name,count_of_vaccinated_patients,mean_length_of_stay
0,Moderna,8,4.60
1,Pfizer,3,4.25
